In [1]:
!pip install --upgrade wandb

Requirement already up-to-date: wandb in c:\users\lloyd\anaconda3\lib\site-packages (0.12.11)


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import wandb
import os
from typing import Any, Dict, List
import copy
import random
import wandb

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_API_KEY"] = "183c1a6a36cbdf0405f5baacb72690845ecc8573"

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

'''mnist = torchvision.datasets.MNIST(root='mnist_data', train=True,download=True, transform=transform)

trainset = torch.utils.data.DataLoader(dataset=mnist,
                                               batch_size=100,
                                               shuffle=True)
testset = torch.utils.data.DataLoader(dataset=mnist,
                                               batch_size=100,
                                               shuffle=True)'''


trainset = torchvision.datasets.CIFAR10(root='cifar_data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='cifar_data', train=False, download=True,transform=transform)

# # This is for iid data
# trainloader = torch.utils.data.DataLoader(trainset, 
#                                           batch_size=500,
#                                           shuffle=True)
testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=100,
                                        shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
n_client = 20
n_leader = int(n_client * 0.1)
chosen_prob = 0.8
local_batch_size = 32
local_epochs = 10

In [5]:
epochs = 10
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD
optimizer_conf = dict(
    lr=0.001,
    momentum=0.9
)

In [6]:
import custom_model
import server
model = custom_model.Net().to(device)
print(n_leader)

server_list: List[server.Server] = []

servers = []


#print(len(server_list))
leaderStep = int(n_client/n_leader)

client_list = []
    
for i in range(0,n_leader):
    server_list = server.Server(
        model=model,
        loss=criteria,
        optimizer=optimizer,
        n_client=int(leaderStep),
        #n_leader=n_leader,
        chosen_prob=chosen_prob,
        optimizer_conf=optimizer_conf,
        local_batch_size=local_batch_size,
        local_epochs=local_epochs
    )
    servers.append(server_list)
    
#add clustering part here and define the leaders (using for loop)
#TODO - generate coordinate for mobility


Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)


Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)


Requirement already up-to-date: wandb in c:\users\lloflo\anaconda3\lib\site-packages (0.12.15)
2


In [35]:

# # This is for iid data
# for batch_idx, (batch_feature, batch_label) in enumerate(trainloader):
#     server.client_pool[batch_idx].setData(list(zip(batch_feature, batch_label)))

# This is for non-iid data
from collections import Counter
import random
import numpy as np
import logging
import clustering
import clients
from scipy.spatial.distance import cdist
from mobility import gauss_markov, reference_point_group, \
    tvc, truncated_levy_walk, random_direction, random_waypoint, random_walk
from tqdm import tqdm

classes_pair = [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]

data_label = np.array(trainset.targets)

nr_nodes = int(n_client)

logger = logging.getLogger("vanet")

step = 0
np.random.seed(0xffff)

# simulation area (units)
MAX_X, MAX_Y = 100, 100

# max and min velocity
MIN_V, MAX_V = 0.1, 1.

# max waiting time
MAX_WT = 100.

# number of steps to ignore before start plotting
STEPS_TO_IGNORE = 10000

# set this to true if you want to calculate node contacts
CALCULATE_CONTACTS = False

# if calculating contacts, this is the range to be used
# (if a distance(a,b) < RANGE, then there is a contact betwen a and b)
RANGE = 1.

rw = random_walk(nr_nodes, dimensions=(MAX_X, MAX_Y))
    
cluster = clustering.Cluster(nr_nodes,step,MAX_X, MAX_Y,MIN_V, MAX_V,MAX_WT,STEPS_TO_IGNORE,CALCULATE_CONTACTS,
                            RANGE,rw)

coordinate = cluster.coord(rw)
#print(coordinate)
coordinates = []

def get_subCoordinates(original_list, number_of_sub_list_wanted):
    sublists = list()
    for sub_list_count in range(number_of_sub_list_wanted): 
        sublists.append(original_list[sub_list_count::number_of_sub_list_wanted])
    return sublists

coordinates = get_subCoordinates(coordinate, n_leader)

print(coordinates)
#print(len(servers[0].client_pool))
#print(coordinate[5][1])
#print(coordinate[2])
#print(cluster.coord(rw))


#initialize coordinates
for i in range(len(servers)):
    for j in range(len(servers[i].client_pool)):
        servers[i].client_pool[j].setXCoordinate(coordinates[i][j][0])
        servers[i].client_pool[j].setYCoordinate(coordinates[i][j][1])

print(str('X coord ')+str(servers[0].client_pool[0].actual_Xcoord))
print(str('Y coord ')+str(servers[0].client_pool[0].actual_Ycoord))

print(str('X coord ')+str(servers[1].client_pool[0].actual_Xcoord))
print(str('Y coord ')+str(servers[1].client_pool[0].actual_Ycoord))
                   
chosen_counter = Counter()
for i in range(len(servers)):
    for _, client in tqdm(enumerate(servers[i].client_pool)):
        # sample until we have a pair of class with insufficient client owning
        class_pair = random.choice(classes_pair)
        while chosen_counter[class_pair] == 100:
            class_pair = random.choice(classes_pair)

        chosen_counter[class_pair] += 1

        first_class, second_class = class_pair
        first_class_sample_idx = list(np.where(data_label == first_class)[0])
        second_class_sample_idx = list(np.where(data_label == second_class)[0])

        client_first_class_sample_idx = random.sample(first_class_sample_idx, k=500)
        client_second_class_sample_idx = random.sample(second_class_sample_idx, k=500)

        client_data = []

        for i in range(500):
            client_data.append(trainset[client_first_class_sample_idx[i]])
            client_data.append(trainset[client_second_class_sample_idx[i]])

        client.setData(client_data)
        #print(client_data)
        #print(client.setData(client_data))print(model.conv1.weight.grad)

'''for i in range(len(servers)):
    for j in range(len(servers[i].client_pool)):
        print(str('For client ')+str(j)+str(' in server ')+str(i)+str(' the lenght of data is ')+str(len(servers[i].client_pool[j].data)))
    print(str('The number of client in server ')+str(i)+str(' is: ')+str(len(servers[i].client_pool)))'''

#print((servers[0].client_pool[0].model))
#print((servers[0].client_pool[1].model))
#print((servers[1].client_pool[2].model))


0it [00:00, ?it/s]

[array([[19.26743972, 25.44610794],
       [90.69726375, 24.51785485],
       [38.88126099, 11.12318674],
       [79.6703566 , 62.46714644],
       [84.86975409, 41.72673228],
       [76.09718472, 65.69433294],
       [19.64789023, 92.65442519],
       [37.17770565, 14.20567892],
       [76.19205742, 69.16390623],
       [38.32178211, 21.67184748]]), array([[95.05819032, 62.00219612],
       [52.27931961, 57.79525622],
       [27.09189255, 57.92444027],
       [33.41449071, 25.42171488],
       [84.50727381, 25.03071363],
       [45.86476139, 66.96604269],
       [20.33964845, 95.06680491],
       [61.87514805,  2.48223398],
       [60.41843274,  1.64728009],
       [64.0899629 , 85.87838921]])]
X coord 19.26743972463429
Y coord 25.44610794302486
X coord 95.05819032330184
Y coord 62.002196122938315


10it [00:05,  1.77it/s]
10it [00:05,  1.77it/s]


"for i in range(len(servers)):\n    for j in range(len(servers[i].client_pool)):\n        print(str('For client ')+str(j)+str(' in server ')+str(i)+str(' the lenght of data is ')+str(len(servers[i].client_pool[j].data)))\n    print(str('The number of client in server ')+str(i)+str(' is: ')+str(len(servers[i].client_pool)))"

In [11]:
wandb.init(project="fl", name="CNN_CIFAR_10_noniid")
#print((servers[0].client_pool[0].data))
aggregates = []

'''for i in range(len(servers)):
    aggregate = servers[i].aggregate()
    aggregates.append(aggregate)'''
    
for i in range(epochs):
    for j in range(len(servers)):
        servers[j].aggregate()
        servers[j].broadcast()
        total_correct = 0
        #with torch.no_grad():
        for _, (test_feature, test_label) in enumerate(testloader):
            test_feature = test_feature.to(device)
            test_label = test_label.to(device)
            y_pred = servers[j].model(test_feature)
            y_pred_decode = torch.argmax(y_pred, dim=1)

            total_correct += y_pred_decode.eq(test_label).sum().item()
            #print(str('total_correct ')+str(total_correct))
        test_acc = total_correct / 10000

        print("Overall acc: {}, overall_loss: {}".format(servers[j].avg_acc, servers[j].avg_loss))
        wandb.log({"acc": servers[j].avg_acc, "loss": servers[j].avg_loss})
#print(model.conv1.weight.grad)

Client 2: Acc 0.8662109375, Loss: 0.26418264117091894
Client 0: Acc 0.7412109375, Loss: 0.501322410069406
Client 6: Acc 0.8671875, Loss: 0.2949364469386637
Client 5: Acc 0.853515625, Loss: 0.29693995718844235
Client 4: Acc 0.873046875, Loss: 0.27076182467862964
Client 9: Acc 0.728515625, Loss: 0.5147119862958789
Client 1: Acc 0.845703125, Loss: 0.33007818879559636
Client 7: Acc 0.8671875, Loss: 0.288080588914454
Overall acc: 0.830322265625, overall_loss: 0.34512675550649874
Client 9: Acc 0.7607421875, Loss: 0.4709209566935897
Client 3: Acc 0.7431640625, Loss: 0.4884306313470006
Client 0: Acc 0.873046875, Loss: 0.26259832084178925
Client 1: Acc 0.7822265625, Loss: 0.40841060830280185
Client 6: Acc 0.7646484375, Loss: 0.46554539166390896
Client 5: Acc 0.7685546875, Loss: 0.45553265884518623
Client 7: Acc 0.8662109375, Loss: 0.2550408111419529
Client 2: Acc 0.7333984375, Loss: 0.5118169486522675
Overall acc: 0.7864990234375, overall_loss: 0.4147870409360621
Client 4: Acc 0.8720703125, Los

Client 5: Acc 0.8720703125, Loss: 0.25847216276451945
Client 1: Acc 0.830078125, Loss: 0.3257129811681807
Client 2: Acc 0.783203125, Loss: 0.3961653672158718
Client 0: Acc 0.9052734375, Loss: 0.18040833808481693
Client 7: Acc 0.9140625, Loss: 0.1651950596133247
Client 4: Acc 0.830078125, Loss: 0.34631034499034286
Client 8: Acc 0.7998046875, Loss: 0.40169224981218576
Client 6: Acc 0.8681640625, Loss: 0.2824100269936025
Overall acc: 0.850341796875, overall_loss: 0.2945458163303556
Client 6: Acc 0.921875, Loss: 0.15022200148086995
Client 8: Acc 0.908203125, Loss: 0.15741279092617333
Client 1: Acc 0.8505859375, Loss: 0.30467555625364184
Client 5: Acc 0.91796875, Loss: 0.15217032458167523
Client 2: Acc 0.9228515625, Loss: 0.12727284617722034
Client 9: Acc 0.8076171875, Loss: 0.3976446585729718
Client 4: Acc 0.904296875, Loss: 0.1784662512363866
Client 3: Acc 0.9130859375, Loss: 0.1707081978674978
Overall acc: 0.893310546875, overall_loss: 0.20482157838705461
Client 9: Acc 0.82421875, Loss: 